# ISCE Processing of SnowEx UAVSAR data
By: **Jack Tarricone** *(University of Nevada, Reno)* \
Guidance from: **Sam Neitlich** *(Montana State/JPL)*, **Yungling Lou** *(JPL)*, **Zach Keskenin** *(Boise State)*, **Ross Palomaki** *(Montana State)*, **Naheem Adebisi** *(Boise State)*

In [3]:
import os
import glob
import numpy as np
import isce
import shutil
#import wget #(fix this)
#import uavsar_pytools
#import matplotlib.pyplot as plt
#import matplotlib
#import rasterio as rio
#from osgeo import gdal
#from rasterio.plot import show

This is the Open Source version of ISCE.
Some of the workflows depend on a separate licensed package.
To obtain the licensed package, please make a request for ISCE
through the website: https://download.jpl.nasa.gov/ops/request/index.cfm.
Alternatively, if you are a member, or can become a member of WinSAR
you may be able to obtain access to a version of the licensed sofware at
https://winsar.unavco.org/software/isce


This tutorial is a recreation of the processing steps outlined [IsceUAVSARprocessing.ipynb](https://github.com/taliboliver/Isce_UAVSAR/blob/master/IsceUAVSARprocessing.ipynb), with addtional functionality added by the ```uavsar_pytools``` package. It is assumed that users have ISCE installed on thier local machines, which admittedly can be the most challenging step in this process.  For information on ISCE installoation please see the [Isce_UAVSAR](https://github.com/taliboliver/Isce_UAVSAR) by **Talib Oliver** *(JPL)* or Eric fieldings (find link from Zach)

In this notebook, we will interfermoterically process SLC stack from the Jemez Mountains, NM using the [Interferometric synthetic aperture radar Scientific Computing Environment (ISCE2)](https://github.com/isce-framework/isce2). We will then phase unwrap using [SNAPHU](https://web.stanford.edu/group/radar/softwareandlinks/sw/snaphu/), and geolocate the SLC data using ```geolocate_uavsar()```.

Past tutorials have assumed that your Jupyter environment will natively run command line code. In this notebook will run ISCE using python functionality.

## Create folder to work out of
First we'll create a new folder in our home directory to download files and run ISCE from

In [4]:
# get home dir
home = '/Users/jacktarricone/'
os.chdir(home)
os.getcwd()

'/Users/jacktarricone'

In [5]:
# working folder name
isce_directory = 'jemez_isce'
    
# path to folder
working_dir = os.path.join(home,isce_directory)

In [6]:
# make new folder
#os.mkdir(working_dir)
print("Directory '%s' created" %isce_directory)

Directory 'jemez_isce' created


In [7]:
# nativate to new folder
os.chdir(working_dir)
os.getcwd()

'/Users/jacktarricone/jemez_isce'

In [15]:
## create subdirectories

# download
dl = 'download'
download_path = os.path.join(working_dir,dl)
#os.mkdir(download_path)

# slc
SLC = 'SLC'
slc_path = os.path.join(working_dir,SLC)
#os.mkdir(slc_path)

# baselines
bl = 'baselines'
baselines_path = os.path.join(working_dir,bl)
#os.mkdir(baselines_path)

# referenceShelve
rs = 'referenceShelve'
referenceShelve_path = os.path.join(working_dir,rs)
#os.mkdir(referenceShelve_path)

# dem
dem = 'DEM'
dem_path = os.path.join(working_dir,dem)
#os.mkdir(dem_path)

# igrams
igrams = 'igrams'
igrams_path = os.path.join(working_dir,igrams)
#os.mkdir(igrams_path)

In [17]:
# check folders were created
for file in os.listdir(working_dir):
    d = os.path.join(working_dir, file)
    if os.path.isdir(d):
        print(d)

/Users/jacktarricone/jemez_isce/SLC
/Users/jacktarricone/jemez_isce/referenceShelve
/Users/jacktarricone/jemez_isce/igrams
/Users/jacktarricone/jemez_isce/download
/Users/jacktarricone/jemez_isce/DEM
/Users/jacktarricone/jemez_isce/baselines


## Data Download

Data links can be found on the UAVSAR site [here](https://uavsar.jpl.nasa.gov/cgi-bin/product.pl?jobName=alamos_35915_01#data"). In this case we'll be using SLCs from Feb 12th and 26th, 2x8 SLC spacing given it's the same as the LLH and LKV files, and just the HH polarization. By selecting these options, the site generates the list of ```wget``` download commands below.

 <p align="left">
  <img width="600" src="imgs/uavsar_data_portal.png">
</p>     
<p style="text-align: center;">
    (Data download screen)
</p>

Here the links are formatted as a string and saved as an exeutable shell file (.sh). This can also be down outside of this notebook by copying the links directly from the UAVSAR data portal and saving into your local text editor.

In [10]:
# nativate to download folder
os.chdir(download_path)
os.getcwd()

'/Users/jacktarricone/jemez_isce/download'

In [11]:
# create .sh file from links

links = ['wget https://downloaduav2.jpl.nasa.gov/Release30/alamos_35915_01/alamos_35915_01_BU.dop',
         'wget https://downloaduav2.jpl.nasa.gov/Release30/alamos_35915_01/alamos_35915_01_BU_s1_2x8.llh',
         'wget https://downloaduav2.jpl.nasa.gov/Release30/alamos_35915_01/alamos_35915_01_BU_s1_2x8.lkv',
         'wget https://downloaduav2.jpl.nasa.gov/Release30/alamos_35915_01/alamos_35915_20005_003_200212_L090HH_01_BU_s1_2x8.slc',
         'wget https://downloaduav2.jpl.nasa.gov/Release30/alamos_35915_01/alamos_35915_20005_003_200212_L090HH_01_BU.ann',
         'wget https://downloaduav2.jpl.nasa.gov/Release30/alamos_35915_01/alamos_35915_20013_000_200226_L090HH_01_BU_s1_2x8.slc',
         'wget https://downloaduav2.jpl.nasa.gov/Release30/alamos_35915_01/alamos_35915_20013_000_200226_L090HH_01_BU.ann']

with open('downloads.sh', 'w') as f:
    for links in links:
        f.write(links)
        f.write('\n')

In [12]:
# ! sh downloads.sh

--2022-06-27 12:40:56--  https://downloaduav2.jpl.nasa.gov/Release30/alamos_35915_01/alamos_35915_01_BU.dop
Resolving downloaduav2.jpl.nasa.gov (downloaduav2.jpl.nasa.gov)... 137.78.249.121
Connecting to downloaduav2.jpl.nasa.gov (downloaduav2.jpl.nasa.gov)|137.78.249.121|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 248175 (242K)
Saving to: ‘alamos_35915_01_BU.dop’

alamos_35915_01_BU. 100%[===================>] 242.36K  1.32MB/s    in 0.2s    

2022-06-27 12:40:57 (1.32 MB/s) - ‘alamos_35915_01_BU.dop’ saved [248175/248175]

--2022-06-27 12:40:57--  https://downloaduav2.jpl.nasa.gov/Release30/alamos_35915_01/alamos_35915_01_BU_s1_2x8.llh
Resolving downloaduav2.jpl.nasa.gov (downloaduav2.jpl.nasa.gov)... 137.78.249.121
Connecting to downloaduav2.jpl.nasa.gov (downloaduav2.jpl.nasa.gov)|137.78.249.121|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 506808720 (483M)
Saving to: ‘alamos_35915_01_BU_s1_2x8.llh’

alamos_35915_01_BU_ 100%

In [20]:
# check folders were created
for i in os.listdir(download_path):
    file = os.path.join(download_path, i)
    print(file)

['alamos_35915_01_BU.dop', 'alamos_35915_20013_000_200226_L090HH_01_BU_s1_2x8.slc', 'alamos_35915_01_BU_s1_2x8.llh', 'alamos_35915_01_BU_s1_2x8.lkv', 'alamos_35915_20005_003_200212_L090HH_01_BU_s1_2x8.slc', 'alamos_35915_20005_003_200212_L090HH_01_BU.ann', 'alamos_35915_20013_000_200226_L090HH_01_BU.ann', 'downloads.sh']
/Users/jacktarricone/jemez_isce/download/alamos_35915_01_BU.dop
/Users/jacktarricone/jemez_isce/download/.DS_Store
/Users/jacktarricone/jemez_isce/download/alamos_35915_20013_000_200226_L090HH_01_BU_s1_2x8.slc
/Users/jacktarricone/jemez_isce/download/alamos_35915_01_BU_s1_2x8.llh
/Users/jacktarricone/jemez_isce/download/alamos_35915_01_BU_s1_2x8.lkv
/Users/jacktarricone/jemez_isce/download/alamos_35915_20005_003_200212_L090HH_01_BU_s1_2x8.slc
/Users/jacktarricone/jemez_isce/download/alamos_35915_20005_003_200212_L090HH_01_BU.ann
/Users/jacktarricone/jemez_isce/download/alamos_35915_20013_000_200226_L090HH_01_BU.ann
/Users/jacktarricone/jemez_isce/download/downloads.sh


In [ ]:
# move doppler file to working directory
shutil.move("alamos_35915_01_BU.dop", "/Users/jacktarricone/jemez_isce/alamos_35915_01_BU.dop")

In [ ]:
! prepareUAVSAR_coregStack_segmets.py -h

In [ ]:
! dem.py